In [13]:
import time
import os
from Tea.exceptions import TeaException
from alibabacloud_tea_openapi import models
from alibabacloud_aiearth_engine20220609.models import *
from alibabacloud_aiearth_engine20220609.client import Client
import aie
import requests
aie.Authenticate(token='')
aie.Initialize()
from aiearth.openapi import build_client
from aiearth.openapi.enums import *
CLIENT = build_client('', '')

计算资源初始化中，请等待...
计算资源初始化完成.


In [ ]:
region = aie.Geometry.Polygon(
    [
        [
            [121.93826854879164, 30.970113616024253],
            [121.92722368998854, 30.969653807534907],
            [121.92926211306607, 30.964191528157762],
            [121.9295732493118, 30.963142738459393],
            [121.92982001254117, 30.96126592814542],
            [121.92984147021329, 30.95840464144593],
            [121.94244755425436, 30.958885338234488],
            [121.95453596983721, 30.962624820837277],
            [121.94630590848263, 30.981625659888707],
            [121.93452564648922, 30.981202542714716]
        ]
    ]
)
 
singleFeature = aie.Feature(region, {'key': 'val'})
ROI = aie.FeatureCollection(singleFeature)           
s2 = aie.ImageCollection('SENTINEL_MSIL2A') \
             .filterBounds(ROI) \
             .filterDate('2023-03-01', '2023-08-30') \
             .filter('eo:cloud_cover<20')\
             .select(['B4', 'B3', 'B2'])
# print(s2.first().propertyNames().getInfo())

# def getID(image):
#     return aie.Feature(None).set({'id': image.id()})
# # 获取id序列，list
# ids = s2.map(getID)
# id_list = ids.aggregate_array('id').getInfo()
# print(id_list)

# 获取时间序列，list
date_list1 = s2.aggregate_array("datetime").getInfo()
date_list  = sorted(set(date_list1))
print(date_list)

# # 定义下载函数
# def export_image(image, roi, file_name):
#     task =  aie.Export.image.toAsset(image, 'LG_'+ date, scale=10)
#     task.start()


# 循环在s2中，filter Date = list[i], first() mean() 
name_list = []  # 用于存储每个任务的 description
for date in date_list:
    # 过滤图像集合以获取特定索引的图像
    image = s2.filter(aie.Filter.eq("datetime", date)).mean().clip(region)
    task_name = 'LG_' + date
    # export_image(image, region, date)
    name_list.append(task_name)
    

In [ ]:
data_id_list=[]
for out_data_id in name_list:
    list_user_raster_req: ListUserRasterDatasRequest = ListUserRasterDatasRequest()
    list_user_raster_req.name = out_data_id
    list_user_raster_req.page_number = 1
    list_user_raster_req.page_size = 10
    list_user_raster_req.from_type = UserDataFromType.DEVELOPER_EXPORT.value
    list_user_raster_resp: ListUserRasterDatasResponse = CLIENT.list_user_raster_datas(list_user_raster_req)
    print(list_user_raster_resp.body.list[0].data_id)
    if list_user_raster_resp.body.list:  # 确保列表不为空
        data_id = list_user_raster_resp.body.list[0].data_id
        data_id_list.append(data_id)
        
    if all([status == PublishStatus.PUBLISHDONE for status in data_id_list]):
        break
    elif any([status == PublishStatus.PUBLISHFAIL for status in data_id_list]):
        raise ValueError(f"Publish Error")
    else:
        time.sleep(3)
print("Data IDs:", data_id_list)

In [ ]:
def sync_create_job(data_id: str) -> (JobStatus, str):
    createAiJobReq: CreateAIJobRequest = CreateAIJobRequest()
    createAiJobReq.job_name = job_name
    createAiJobReqInputSrc = CreateAIJobRequestInputsSrc()
    createAiJobReqInputSrc.data_id = data_id
    createAiJobReqInput = CreateAIJobRequestInputs()
    createAiJobReqInput.src = createAiJobReqInputSrc
    createAiJobReqInput.idx = 1
    createAiJobReq.inputs = [createAiJobReqInput]
    createAiJobReq.shape_data_id ='7a9afac4038bdd4b9120b21a5d5c3652'
    createAiJobReq.app = 'S2_SUPER_RESOLUTION'
    createAiJobReq.confidence = 30
    createAiJobReq.area_threshold = 1

    createAiJobResp: CreateAIJobResponse = CLIENT.create_aijob(createAiJobReq)
    job_id = createAiJobResp.body.jobs[0].job_id

    while True:
        get_job_req: GetJobsRequest = GetJobsRequest()
        get_job_req.job_ids = [job_id]

        get_job_resp: GetJobsResponse = CLIENT.get_jobs(get_job_req)
        job_status = JobStatus(get_job_resp.body.list[0].status)
        print(f"JobId: {job_id}, jobStatus: {job_status}")
        if job_status == JobStatus.FINISHED:
            out_data_id = get_job_resp.body.list[0].job_out_data_id
            out_data_type = JobOutDataType(get_job_resp.body.list[0].out_data_type)
            break
        elif job_status == JobStatus.ERROR:
            raise ValueError(f"{job_id} error")
        else:
            time.sleep(500)
            
    return job_status, out_data_id

In [ ]:
for index, data_id in enumerate(data_id_list):
    # job_name = f'image{index + 1}'  # 生成不同的任务名称，如 image1、image2、image3
    image = aie.Image(f'user/{data_id}')
    datetime_value = image.get('title').getInfo()
    job_name = f'Super_resolution_{datetime_value[10:19] }'
    # job_name = f'LG_super_resolution_{name}'
    job_status, out_data_id = sync_create_job(data_id)
    